In [ ]:
!pip install tensorflow numpy pandas yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Download historical stock data
ticker = 'ADANIENSOL.NS'
data = yf.download(ticker)
prices = data['Close'].values

# Normalize the data
scaler = MinMaxScaler()
prices = scaler.fit_transform(prices.reshape(-1, 1))

# Prepare sequences for the CNN-LSTM model
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 10
X, y = create_sequences(prices, seq_length)

# Split the data into training and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape X for CNN-LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

[*********************100%%**********************]  1 of 1 completed


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(seq_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 9, 64)             192       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 4, 64)             0         
 g1D)                                                            
                                                                 
 lstm_1 (LSTM)               (None, 50)                23000     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 23243 (90.79 KB)
Trainable params: 23243 (90.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
6/6 [==============================] - 4s 89ms/step - loss: 0.3238 - mae: 0.4949 - val_loss: 0.3117 - val_mae: 0.5510
Epoch 2/50
6/6 [==============================] - 0s 22ms/step - loss: 0.2206 - mae: 0.4028 - val_loss: 0.1968 - val_mae: 0.4347
Epoch 3/50
6/6 [==============================] - 0s 18ms/step - loss: 0.1333 - mae: 0.3051 - val_loss: 0.0917 - val_mae: 0.2893
Epoch 4/50
6/6 [==============================] - 0s 28ms/step - loss: 0.0577 - mae: 0.2002 - val_loss: 0.0167 - val_mae: 0.1075
Epoch 5/50
6/6 [==============================] - 0s 27ms/step - loss: 0.0218 - mae: 0.1147 - val_loss: 0.0215 - val_mae: 0.1166
Epoch 6/50
6/6 [==============================] - 0s 21ms/step - loss: 0.0302 - mae: 0.1535 - val_loss: 0.0155 - val_mae: 0.0909
Epoch 7/50
6/6 [==============================] - 0s 27ms/step - loss: 0.0202 - mae: 0.1126 - val_loss: 0.0104 - val_mae: 0.0731
Epoch 8/50
6/6 [==============================] - 0s 26ms/step - loss: 0.0199 - mae: 0.1091 - val

In [ ]:
# Function to predict future steps
def predict_future_steps(model, data, steps):
    predictions = []
    input_seq = data[-seq_length:].reshape((1, seq_length, 1))
    for _ in range(steps):
        pred = model.predict(input_seq)
        predictions.append(pred[0, 0])
        input_seq = np.append(input_seq[:, 1:, :], pred.reshape(1, 1, 1), axis=1)
    return np.array(predictions)

# Predict the next day and week prices
next_1d_pred = predict_future_steps(model, prices, 1)
next_1w_pred = predict_future_steps(model, prices, 7)

# Transform the predictions back to the original scale
next_1d_pred = scaler.inverse_transform(next_1d_pred.reshape(-1, 1))
next_1w_pred = scaler.inverse_transform(next_1w_pred.reshape(-1, 1))

1/1 [==============================] - 0s 32ms/step


In [ ]:
print(f'1-Day Forecast: {next_1d_pred.flatten()}')
print(f'1-Week Forecast: {next_1w_pred.flatten()}')

1-Day Forecast: [1035.9385]
1-Week Forecast: [1035.9385 1081.4739 1084.1753 1086.2109 1087.0728 1077.9159 1080.1122]
